In [1]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image, ImageFile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [2]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# keras imports
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Input
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

Using TensorFlow backend.


In [3]:
tf.__version__

'1.13.2'

In [4]:
# Specify image size
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3

In [5]:
def getRadius(img):
#     print(type(img.shape))
    circles = cv2.HoughCircles(img,3,1,max(img.shape)/2,param1=50,param2=30,minRadius= int(max(img.shape)/6),maxRadius=max(img.shape))
    return circles[0][0][2]
def get_img(id_code):
    img_path = PATH_IMG+id_code+'.png'
    img = cv2.imread(img_path)
#     plt.imshow(img)
    return img

In [6]:
from imgaug import augmenters as ia
import os
import matplotlib.pyplot as plt

PATH_IN =  'data/tr'
PATH_IMG = PATH_IN + "data/train/"

#read in table with image ids and retinopathy severity rating for associated image
df = pd.read_csv("train.csv")
# df = df.head(20).copy()
gc.collect()

# idiot-test with first image
id_code = df['id_code'].values[0]
label = df['diagnosis'].values[0]

In [7]:
#instantiate pre-trained ResNet model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
res_fifty = keras.models.model_from_json(loaded_model_json)
# load weights into new model
res_fifty.load_weights("model.h5")

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
pd.set_option('max_colwidth', -1)

layers = [(layer, layer.name, layer.trainable) for layer in res_fifty.layers]
df_layers = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
df_layers

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f24eed1e9b0>,input_1,True
1,<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f24eed1ee10>,conv1_pad,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f24eed34438>,conv1,False
3,<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7f24eed34898>,bn_conv1,False
4,<tensorflow.python.keras.layers.core.Activation object at 0x7f24eed34c18>,activation,False
5,<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f24eed34da0>,pool1_pad,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f24eed34f60>,max_pooling2d,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f24eed56240>,res2a_branch2a,False
8,<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7f24eed566a0>,bn2a_branch2a,False
9,<tensorflow.python.keras.layers.core.Activation object at 0x7f24eed56a20>,activation_1,False


In [37]:
# from keras import optimizers
datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range = 180,horizontal_flip = True, vertical_flip = True, data_format = 'channels_last')
batch_size = 20
# datagen.fit(ordered_imgs)
 
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode = 'categorical')


Found 32493 images belonging to 5 classes.


In [38]:
# from keras import backend as K
get_last_pretrained_layer = keras.backend.function([res_fifty.layers[0].input, keras.backend.learning_phase()],
                                  [res_fifty.layers[176].output])


In [39]:
train_features = np.zeros(shape=(32493, 512))
train_labels = np.zeros(shape=(32493,5))

In [41]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = get_last_pretrained_layer(inputs_batch)[0]
    train_features[i : i + len(inputs_batch)] = features_batch 
    train_labels[i : i +len(inputs_batch)] = labels_batch
    i+= len(inputs_batch)
#     print(i)
    if i+2 > train_features.shape[0]:
        print('beep')
        break
# train_features = np.reshape(train_features, (len(ordered_imgs), 7 * 7 * 512))

ValueError: could not broadcast input array from shape (20,512) into shape (19,512)

In [15]:
del train_generator; gc.collect()

299

In [42]:
i,j = np.where(train_labels==1)

In [43]:
len(j)

31033

In [44]:
sub = train_features[:len(j),:]

In [45]:
sub.shape

(31033, 512)

In [46]:
import xgboost as xgb
from sklearn.svm import SVC as svc
from sklearn.model_selection import train_test_split
# train_features = np.float32(train_features)
# params = {'objective':'multi:softprob'}
features_train, features_test, labels_train, labels_test = train_test_split(sub,j,test_size = 0.2)
clf = svc(kernel = 'poly')
clf.fit(features_train,labels_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='poly', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [19]:
import pickle
s = pickle.dumps(clf)

In [ ]:
a = pickle.dumps(clf)

In [47]:
preds = clf.predict(features_test)

In [49]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(labels_test,preds, weights = 'quadratic')

0.6759165118919704

polynomial kernel in svm gets higher cohen's kappa than linear kernel and xgboost with "balanced" classes

In [34]:
del train_features, features_train, labels_train; gc.collect()

197

In [21]:
submit = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_imgs = []
names = {}

In [22]:
from tqdm import tqdm
import matplotlib.pyplot as plt
predicted = []
for i, name in tqdm(enumerate(submit['id_code'])):
    path = os.path.join('../input/aptos2019-blindness-detection/test_images/', name+'.png')
    img_array = cv2.imread(path)
    #crop each image to 80% of fundus diameter
    try:
        r = getRadius(img_array)
        crop_r = round(r*0.8) 
    except:
            crop_r = (img_array.shape[1]/2)*0.8
            
    center_x = int(img_array.shape[1]/2)
    center_y = int(img_array.shape[0]/2)
    
    left_border = int(center_x - crop_r)
    right_border = int(center_x + crop_r)
    upper_border = int(center_y - crop_r)
    lower_border = int(center_y + crop_r)
    
    if left_border < 0:
            difference = 0- left_border
            left_border = 0
            right_border -=difference
            upper_border += difference
            lower_border -= difference
    if upper_border < 0:
            difference = 0- upper_border
            upper_border = 0
            lower_border -= difference
            left_border += difference
            right_border -= difference

    a = cv2.resize(img_array, (224,224), interpolation = cv2.INTER_CUBIC)
    img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    
    test_imgs.append(img)
    score_predict = clf.predict(get_last_pretrained_layer(np.asarray(test_imgs[0:1]))[0])
    predicted.append(score_predict[0])
    test_imgs.pop()

2it [00:00,  6.60it/s]

1
2


4it [00:00,  6.84it/s]

2
2


6it [00:00,  6.96it/s]

...

0


1926it [05:16,  6.06it/s]

1
1


1928it [05:17,  5.87it/s]

2
0


In [23]:
submit['diagnosis'] = predicted
submit.to_csv('submission.csv', index=False)
submit.head()

,id_code,diagnosis
0,0005cfc8afb6,1
1,003f0afdcd15,2
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,4


occ produces 0.63 accuracy w/ 3662//5 train images, frozen weights

0.68 w/ all train images, last 20 layers unfrozen

0.58 w/ xgboost

0.39 w/ xgboost regressor, Ben's preprocessing algorithm